<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>PROPHET - Clase 05</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:20px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>y(t) = g(t) + s(t) + h(t) + e(t)</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

<a id = "1.0"></a>
<p style="font-size:12px;color:#16268a;font-family:'Avantgarde';text-align:right;border-radius:5px;">
<strong>
<a href="https://ichi.pro/es/analisis-de-series-de-tiempo-con-facebook-prophet-como-funciona-y-como-usarlo-265389504990778/" target=\"_blank\">Análisis de series de tiempo</a>
</strong></p>
<p style="font-size:12px;color:#16268a;font-family:'Avantgarde';text-align:right;border-radius:5px;">
<strong>
<a href="https://facebook.github.io/prophet/docs/quick_start.html" target=\"_blank\">FBProphet Help</a>
</strong></p>


In [3]:
import logging
logging.getLogger('prophet').setLevel(logging.WARNING) 

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,7)

In [5]:
import pandas as pd
#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib') 
pd.options.mode.chained_assignment = None

# En produccion eliminar, se incluye por informacion
print(f'Pandas Version: {pd.__version__}') 

Pandas Version: 2.2.2


In [6]:
# Lee el archivo como un DataFrame
df  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(df.head())
print(df_pid_validos.head())

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn     tn
0   201701        10234       20524                      0                 2            0.053  0.053
1   201701        10032       20524                      0                 1            0.136  0.136
2   201701        10217       20524                      0                 1            0.030  0.030
3   201701        10125       20524                      0                 1            0.023  0.023
4   201701        10012       20524                      0                11            1.545  1.545
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2945818 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   periodo                int64  
 1   customer_id            int64  
 2   product_id             int64  
 3   plan_precios_cuidados  int64  
 4   cust_request_qty       int64  
 5   cust_request_tn        float64
 6   tn                     float64
dtypes: float64(2), int64(5)
memory usage: 157.3 MB


In [8]:
df.sample(5)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
560809,201707,10152,20200,0,1,0.044,0.044
403968,201705,10189,20239,0,2,0.035,0.035
1041012,201712,10462,20650,0,1,0.010,0.010
2248128,201903,10356,20428,0,1,0.002,0.002
1044335,201712,10438,20848,0,1,0.005,0.005


In [9]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

In [10]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
# En un nuevo Data Frame llamado "DATA"
data = df[df['product_id'].isin(product_ids_validos)]

In [11]:
data.periodo = pd.to_datetime(data.periodo, format='%Y%m')
# Convertir 'periodo' a formato de fecha
#data['periodo'] = pd.to_datetime(data['periodo'], format='%Y%m')

In [12]:
data.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,2017-01-01,10234,20524,0,2,0.053,0.053
1,2017-01-01,10032,20524,0,1,0.136,0.136
2,2017-01-01,10217,20524,0,1,0.030,0.030
3,2017-01-01,10125,20524,0,1,0.023,0.023
4,2017-01-01,10012,20524,0,11,1.545,1.545


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2293481 entries, 0 to 2945817
Data columns (total 7 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   periodo                datetime64[ns]
 1   customer_id            int64         
 2   product_id             int64         
 3   plan_precios_cuidados  int64         
 4   cust_request_qty       int64         
 5   cust_request_tn        float64       
 6   tn                     float64       
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 140.0 MB


<a id = "1.0"></a>
<p style="font-size:17px;color:#34656d;font-family:'Comic Sans MS';text-align:left;border-radius:5px;">
<strong>Configuración para Prophet</strong></p> 

In [ ]:
# Paso 3: Preparación de los Datos
# Prophet requiere que los datos tengan las columnas ds (para la fecha) y y (para el valor). Organizaremos los datos en este formato.

In [14]:
# Crear un diccionario para almacenar las series temporales por product_id
series_dict = {product_id: df[df['product_id'] == product_id][['periodo', 'tn']].rename(columns={'periodo': 'ds', 'tn': 'y'})
               for product_id in df['product_id'].unique()}

In [15]:
# Paso 4: Entrenamiento del Modelo
# Entrenaremos un modelo Prophet para cada product_id.

In [19]:
# Iterar sobre cada serie temporal
for product_id, series in series_dict.items():
    # Asegurarse de que la columna 'ds' esté en el formato correcto
    series['ds'] = pd.to_datetime(series['ds'])
    
    # Entrenar un modelo Prophet
    model = Prophet()
    model.fit(series)
    
    # Hacer una predicción para varios meses en el futuro
    future = model.make_future_dataframe(periods=2, freq='MS')  # Crear un DataFrame para los próximos 2 meses
    forecast = model.predict(future)
    
    # Imprimir las fechas generadas para depuración
    print(f"Fechas generadas para product_id {product_id}:")
    print(forecast['ds'])
    
    # Buscar la fecha más cercana a febrero de 2020
    pred_date = pd.Timestamp('2020-02-01')
    closest_date = forecast.iloc[(forecast['ds'] - pred_date).abs().argsort()[:1]]['ds'].values[0]
    pred_tn = forecast[forecast['ds'] == closest_date]['yhat'].values[0]
    
    predictions.append([product_id, pred_tn])

# Crear un DataFrame con las predicciones
pred_df = pd.DataFrame(predictions, columns=['product_id', 'tn'])

19:21:00 - cmdstanpy - INFO - Chain [1] start processing
19:21:01 - cmdstanpy - INFO - Chain [1] done processing


Fechas generadas para product_id 20524:
0    1970-01-01 00:00:00.000201701
1    1970-01-01 00:00:00.000201702
2    1970-01-01 00:00:00.000201703
3    1970-01-01 00:00:00.000201704
4    1970-01-01 00:00:00.000201705
5    1970-01-01 00:00:00.000201706
6    1970-01-01 00:00:00.000201707
7    1970-01-01 00:00:00.000201708
8    1970-01-01 00:00:00.000201709
9    1970-01-01 00:00:00.000201710
10   1970-01-01 00:00:00.000201711
11   1970-01-01 00:00:00.000201712
12   1970-01-01 00:00:00.000201801
13   1970-01-01 00:00:00.000201802
14   1970-01-01 00:00:00.000201803
15   1970-01-01 00:00:00.000201804
16   1970-01-01 00:00:00.000201805
17   1970-01-01 00:00:00.000201806
18   1970-01-01 00:00:00.000201807
19   1970-01-01 00:00:00.000201808
20   1970-01-01 00:00:00.000201809
21   1970-01-01 00:00:00.000201810
22   1970-01-01 00:00:00.000201811
23   1970-01-01 00:00:00.000201812
24   1970-01-01 00:00:00.000201901
25   1970-01-01 00:00:00.000201902
26   1970-01-01 00:00:00.000201903
27   1970-01-01

19:21:01 - cmdstanpy - INFO - Chain [1] start processing
19:21:02 - cmdstanpy - INFO - Chain [1] done processing


Fechas generadas para product_id 20311:
0    1970-01-01 00:00:00.000201701
1    1970-01-01 00:00:00.000201702
2    1970-01-01 00:00:00.000201703
3    1970-01-01 00:00:00.000201704
4    1970-01-01 00:00:00.000201705
5    1970-01-01 00:00:00.000201706
6    1970-01-01 00:00:00.000201707
7    1970-01-01 00:00:00.000201708
8    1970-01-01 00:00:00.000201709
9    1970-01-01 00:00:00.000201710
10   1970-01-01 00:00:00.000201711
11   1970-01-01 00:00:00.000201712
12   1970-01-01 00:00:00.000201801
13   1970-01-01 00:00:00.000201802
14   1970-01-01 00:00:00.000201803
15   1970-01-01 00:00:00.000201804
16   1970-01-01 00:00:00.000201805
17   1970-01-01 00:00:00.000201806
18   1970-01-01 00:00:00.000201807
19   1970-01-01 00:00:00.000201808
20   1970-01-01 00:00:00.000201809
21   1970-01-01 00:00:00.000201810
22   1970-01-01 00:00:00.000201811
23   1970-01-01 00:00:00.000201812
24   1970-01-01 00:00:00.000201901
25   1970-01-01 00:00:00.000201902
26   1970-01-01 00:00:00.000201903
27   1970-01-01

19:21:03 - cmdstanpy - INFO - Chain [1] start processing
19:21:04 - cmdstanpy - INFO - Chain [1] done processing


...

Fechas generadas para product_id 20484:
0    1970-01-01 00:00:00.000201701
1    1970-01-01 00:00:00.000201702
2    1970-01-01 00:00:00.000201703
3    1970-01-01 00:00:00.000201704
4    1970-01-01 00:00:00.000201705
5    1970-01-01 00:00:00.000201706
6    1970-01-01 00:00:00.000201707
7    1970-01-01 00:00:00.000201708
8    1970-01-01 00:00:00.000201709
9    1970-01-01 00:00:00.000201710
10   1970-01-01 00:00:00.000201711
11   1970-01-01 00:00:00.000201712
12   1970-01-01 00:00:00.000201801
13   1970-01-01 00:00:00.000201802
14   1970-01-01 00:00:00.000201803
15   1970-01-01 00:00:00.000201804
16   1970-01-01 00:00:00.000201805
17   1970-01-01 00:00:00.000201806
18   1970-01-01 00:00:00.000201807
19   1970-01-01 00:00:00.000201808
20   1970-01-01 00:00:00.000201809
21   1970-01-01 00:00:00.000201810
22   1970-01-01 00:00:00.000201811
23   1970-01-01 00:00:00.000201812
24   1970-01-01 00:00:00.000201901
25   1970-01-01 00:00:00.000201902
26   1970-01-01 00:00:00.000201903
27   1970-01-01

19:21:38 - cmdstanpy - INFO - Chain [1] start processing
19:21:39 - cmdstanpy - INFO - Chain [1] done processing


Fechas generadas para product_id 20641:
0    1970-01-01 00:00:00.000201701
1    1970-01-01 00:00:00.000201702
2    1970-01-01 00:00:00.000201703
3    1970-01-01 00:00:00.000201704
4    1970-01-01 00:00:00.000201705
5    1970-01-01 00:00:00.000201706
6    1970-01-01 00:00:00.000201707
7    1970-01-01 00:00:00.000201708
8    1970-01-01 00:00:00.000201709
9    1970-01-01 00:00:00.000201710
10   1970-01-01 00:00:00.000201711
11   1970-01-01 00:00:00.000201712
12   1970-01-01 00:00:00.000201801
13   1970-01-01 00:00:00.000201802
14   1970-01-01 00:00:00.000201803
15   1970-01-01 00:00:00.000201804
16   1970-01-01 00:00:00.000201805
17   1970-01-01 00:00:00.000201806
18   1970-01-01 00:00:00.000201807
19   1970-01-01 00:00:00.000201808
20   1970-01-01 00:00:00.000201809
21   1970-01-01 00:00:00.000201810
22   1970-01-01 00:00:00.000201811
23   1970-01-01 00:00:00.000201812
24   1970-01-01 00:00:00.000201901
25   1970-01-01 00:00:00.000201902
26   1970-01-01 00:00:00.000201903
27   1970-01-01

19:21:39 - cmdstanpy - INFO - Chain [1] start processing
19:21:40 - cmdstanpy - INFO - Chain [1] done processing
19:21:40 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted


RuntimeError: Error during optimization! Command 'C:\Users\hbasilio\Documents\PERSONAL\Maestria\Materias\LABO_3\ambiente-virtual\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=83526 data file=C:\Users\hbasilio\AppData\Local\Temp\tmp8rqlazem\pc9nwvux.json init=C:\Users\hbasilio\AppData\Local\Temp\tmp8rqlazem\j1m9w6k2.json output file=C:\Users\hbasilio\AppData\Local\Temp\tmp8rqlazem\prophet_modelwqt8qqig\prophet_model-20240605192139.csv method=optimize algorithm=newton iter=10000' failed: console log output:

method = optimize
  optimize
    algorithm = newton
      newton
    jacobian = 0 (Default)
    iter = 10000
    save_iterations = 0 (Default)
id = 1 (Default)
data
  file = C:\Users\hbasilio\AppData\Local\Temp\tmp8rqlazem\pc9nwvux.json
init = C:\Users\hbasilio\AppData\Local\Temp\tmp8rqlazem\j1m9w6k2.json
random
  seed = 83526
output
  file = C:\Users\hbasilio\AppData\Local\Temp\tmp8rqlazem\prophet_modelwqt8qqig\prophet_model-20240605192139.csv
  diagnostic_file 

In [ ]:
# Paso 5: Exportación a CSV
# Guardaremos los resultados en un archivo CSV.

In [ ]:
# Guardar el DataFrame de predicciones en un archivo CSV
pred_df.to_csv("PROPHET_v3.0.predicciones_febrero_2020.csv", index=False)

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 